<a href="https://colab.research.google.com/github/PraveenKumarGarlapati/AV_LoanDelinquencyAug15th/blob/master/Entity%20detection%20and%20validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [182]:
df = pd.read_excel('/content/entity_2045.xlsx')
df.head()

,ID_SID,Sentence,entity,descriptor,GCP_Entities,PerfectMatch
0,BKG-20190696110_1,comfortable room,room,comfortable,['room'],room
1,BKG-20190696110_2,great location,location,great,['location'],location
2,BKG-20190696110_3,good facilities,facilities,good,['facilities'],facilities
3,BKG-201909713799_1,very good location great staff and great exper...,location,good,"['location', 'staff', 'experience']",location
4,BKG-201909713799_1,very good location great staff and great exper...,staff,great,"['location', 'staff', 'experience']",staff


In [0]:
df.GCP_Entities = df.GCP_Entities.apply(lambda x: str(x.replace("'","").replace("[","").replace("]","")).strip())

In [0]:
df['PerfectMatch_GCP'] = df.GCP_Entities.apply(phrase_matching)

In [191]:
from textblob import TextBlob
import nltk
nltk.download('brown')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


df['TB_entities'] = df.Sentence.apply(lambda x : [a for a,b in TextBlob(x).pos_tags if b in ['NN','NNS','NNP','NNPS']])
df

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,ID_SID,Sentence,entity,descriptor,GCP_Entities,PerfectMatch,PerfectMatch_GCP,TB_entities
0,BKG-20190696110_1,comfortable room,room,comfortable,room,room,[room],[room]
1,BKG-20190696110_2,great location,location,great,location,location,[location],[location]
2,BKG-20190696110_3,good facilities,facilities,good,facilities,facilities,[facilities],[facilities]
3,BKG-201909713799_1,very good location great staff and great exper...,location,good,"location, staff, experience",location,"[location, staff, experience]","[location, staff, experience, time]"
4,BKG-201909713799_1,very good location great staff and great exper...,staff,great,"location, staff, experience",staff,"[location, staff, experience]","[location, staff, experience, time]"
...,...,...,...,...,...,...,...,...
2084,GGL-20190685608_3,the breakfast was fine if you like a cooked br...,croissant pastry,NaN,"breakfast, breakfast, lot, croissant pastry li...",NaN,"[breakfast, breakfast, pastry, offer]","[breakfast, breakfast, lot, offer, pastry, line]"
2085,GGL-20190685608_4,also the bananas were very over ripe,bananas,over ripe,bananas,bananas,[bananas],"[bananas, ripe]"
2086,GGL-20190685608_5,and most importantly no de caff tea bags,tea bags,no,caff tea bags,NaN,"[tea, tea bags]","[tea, bags]"
2087,GGL-20190685608_6,however overall a great stay for a decent price,stay,great,"stay, price",stay,"[stay, price]","[stay, price]"


In [0]:
for index, i in enumerate(df.entity):
    if str(i) in df.TB_entities[index]:
        df.loc[index, 'PerfectMatch_TB'] = i
    else:
        df.loc[index, 'PerfectMatch_TB'] = None 

In [193]:
df.isna().sum()

ID_SID                0
Sentence              0
entity               83
descriptor          778
GCP_Entities          0
PerfectMatch        686
PerfectMatch_GCP      0
TB_entities           0
PerfectMatch_TB     701
dtype: int64

In [194]:
df

,ID_SID,Sentence,entity,descriptor,GCP_Entities,PerfectMatch,PerfectMatch_GCP,TB_entities,PerfectMatch_TB
0,BKG-20190696110_1,comfortable room,room,comfortable,room,room,[room],[room],room
1,BKG-20190696110_2,great location,location,great,location,location,[location],[location],location
2,BKG-20190696110_3,good facilities,facilities,good,facilities,facilities,[facilities],[facilities],facilities
3,BKG-201909713799_1,very good location great staff and great exper...,location,good,"location, staff, experience",location,"[location, staff, experience]","[location, staff, experience, time]",location
4,BKG-201909713799_1,very good location great staff and great exper...,staff,great,"location, staff, experience",staff,"[location, staff, experience]","[location, staff, experience, time]",staff
...,...,...,...,...,...,...,...,...,...
2084,GGL-20190685608_3,the breakfast was fine if you like a cooked br...,croissant pastry,NaN,"breakfast, breakfast, lot, croissant pastry li...",NaN,"[breakfast, breakfast, pastry, offer]","[breakfast, breakfast, lot, offer, pastry, line]",None
2085,GGL-20190685608_4,also the bananas were very over ripe,bananas,over ripe,bananas,bananas,[bananas],"[bananas, ripe]",bananas
2086,GGL-20190685608_5,and most importantly no de caff tea bags,tea bags,no,caff tea bags,NaN,"[tea, tea bags]","[tea, bags]",None
2087,GGL-20190685608_6,however overall a great stay for a decent price,stay,great,"stay, price",stay,"[stay, price]","[stay, price]",stay


In [195]:
import spacy
nlp = spacy.load('en_core_web_sm')
df['Sp_entities'] = df.Sentence.apply(lambda x : [i.text for i in nlp(x) if i.tag_ in ['NN','NNS','NNP','NNPS']])
df

,ID_SID,Sentence,entity,descriptor,GCP_Entities,PerfectMatch,PerfectMatch_GCP,TB_entities,PerfectMatch_TB,Sp_entities
0,BKG-20190696110_1,comfortable room,room,comfortable,room,room,[room],[room],room,[room]
1,BKG-20190696110_2,great location,location,great,location,location,[location],[location],location,[location]
2,BKG-20190696110_3,good facilities,facilities,good,facilities,facilities,[facilities],[facilities],facilities,[facilities]
3,BKG-201909713799_1,very good location great staff and great exper...,location,good,"location, staff, experience",location,"[location, staff, experience]","[location, staff, experience, time]",location,"[location, staff, experience, time]"
4,BKG-201909713799_1,very good location great staff and great exper...,staff,great,"location, staff, experience",staff,"[location, staff, experience]","[location, staff, experience, time]",staff,"[location, staff, experience, time]"
...,...,...,...,...,...,...,...,...,...,...
2084,GGL-20190685608_3,the breakfast was fine if you like a cooked br...,croissant pastry,NaN,"breakfast, breakfast, lot, croissant pastry li...",NaN,"[breakfast, breakfast, pastry, offer]","[breakfast, breakfast, lot, offer, pastry, line]",None,"[breakfast, breakfast, lot, offer, pastry, line]"
2085,GGL-20190685608_4,also the bananas were very over ripe,bananas,over ripe,bananas,bananas,[bananas],"[bananas, ripe]",bananas,[bananas]
2086,GGL-20190685608_5,and most importantly no de caff tea bags,tea bags,no,caff tea bags,NaN,"[tea, tea bags]","[tea, bags]",None,"[tea, bags]"
2087,GGL-20190685608_6,however overall a great stay for a decent price,stay,great,"stay, price",stay,"[stay, price]","[stay, price]",stay,"[stay, price]"


In [0]:
for index, i in enumerate(df.entity):
    if str(i) in df.Sp_entities[index]:
        df.loc[index, 'PerfectMatch_Sp'] = i
    else:
        df.loc[index, 'PerfectMatch_TB'] = None 

In [197]:
df.isna().sum()

ID_SID                0
Sentence              0
entity               83
descriptor          778
GCP_Entities          0
PerfectMatch        686
PerfectMatch_GCP      0
TB_entities           0
PerfectMatch_TB     730
Sp_entities           0
PerfectMatch_Sp     690
dtype: int64

In [0]:
##Muthus's final list of entities
ent_cleanup = pd.read_excel('/content/entity clean up - v2.xlsx')


In [0]:
ent_list = ent_cleanup['Propsed Final List of Entities (Col A minus Col B PLUS Col D)'].dropna()

In [0]:
#Find the entities from this list
import spacy
nlp = spacy.load('en_core_web_sm')
from spacy.matcher import PhraseMatcher
matcher = PhraseMatcher(nlp.vocab)


from spacy.matcher import PhraseMatcher
matcher = PhraseMatcher(nlp.vocab, attr='LOWER')
# the list containing the pharses to be matched
terminology_list = ent_list
# convert the phrases into document object using nlp.make_doc to #speed up.
patterns = [nlp.make_doc(text) for text in terminology_list]
# add the patterns to the matcher object without any callbacks
matcher.add("Phrase Matching", None, *patterns)
# the input text string is converted to a Document object

def phrase_matching(text):

  doc = nlp(text)
  #call the matcher object the document object and it will return #match_id, start and stop indexes of the matched words
  matches = matcher(doc)
  #print the matched results and extract out the results
  list1 = []
  for match_id, start, end in matches:
      
      # Get the string representation 
      string_id = nlp.vocab.strings[match_id]  
      span = doc[start:end]  # The matched span
      #print(match_id, string_id, start, end, span.text)
      list1.append(span.text)
  return list1

In [0]:
df['Entity_CleanupFile'] = df.Sentence.apply(phrase_matching)

In [0]:
for index, i in enumerate(df.entity):
    if str(i) in df.Entity_CleanupFile[index]:
        df.loc[index, 'PerfectMatch_EntityCleanupFile'] = i
    else:
        df.loc[index, 'PerfectMatch_EntityCleanupFile'] = None 


In [203]:
df.isna().sum()

ID_SID                              0
Sentence                            0
entity                             83
descriptor                        778
GCP_Entities                        0
PerfectMatch                      686
PerfectMatch_GCP                    0
TB_entities                         0
PerfectMatch_TB                   730
Sp_entities                         0
PerfectMatch_Sp                   690
Entity_CleanupFile                  0
PerfectMatch_EntityCleanupFile    479
dtype: int64

In [204]:
df.columns

Index(['ID_SID', 'Sentence', 'entity', 'descriptor', 'GCP_Entities',
       'PerfectMatch', 'PerfectMatch_GCP', 'TB_entities', 'PerfectMatch_TB',
       'Sp_entities', 'PerfectMatch_Sp', 'Entity_CleanupFile',
       'PerfectMatch_EntityCleanupFile'],
      dtype='object')

In [216]:
df1 = df[['ID_SID', 'Sentence', 'entity', 'PerfectMatch','PerfectMatch_TB', 
       'PerfectMatch_Sp','PerfectMatch_EntityCleanupFile','PerfectMatch_GCP']]

df

,ID_SID,Sentence,entity,descriptor,GCP_Entities,PerfectMatch,PerfectMatch_GCP,TB_entities,PerfectMatch_TB,Sp_entities,PerfectMatch_Sp,Entity_CleanupFile,PerfectMatch_EntityCleanupFile
0,BKG-20190696110_1,comfortable room,room,comfortable,room,room,[room],[room],room,[room],room,[room],room
1,BKG-20190696110_2,great location,location,great,location,location,[location],[location],location,[location],location,[location],location
2,BKG-20190696110_3,good facilities,facilities,good,facilities,facilities,[facilities],[facilities],facilities,[facilities],facilities,[facilities],facilities
3,BKG-201909713799_1,very good location great staff and great exper...,location,good,"location, staff, experience",location,"[location, staff, experience]","[location, staff, experience, time]",location,"[location, staff, experience, time]",location,"[location, staff, experience]",location
4,BKG-201909713799_1,very good location great staff and great exper...,staff,great,"location, staff, experience",staff,"[location, staff, experience]","[location, staff, experience, time]",staff,"[location, staff, experience, time]",staff,"[location, staff, experience]",staff
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2084,GGL-20190685608_3,the breakfast was fine if you like a cooked br...,croissant pastry,NaN,"breakfast, breakfast, lot, croissant pastry li...",NaN,"[breakfast, breakfast, pastry, offer]","[breakfast, breakfast, lot, offer, pastry, line]",None,"[breakfast, breakfast, lot, offer, pastry, line]",NaN,"[breakfast, breakfast, offer, pastry]",None
2085,GGL-20190685608_4,also the bananas were very over ripe,bananas,over ripe,bananas,bananas,[bananas],"[bananas, ripe]",bananas,[bananas],bananas,[bananas],bananas
2086,GGL-20190685608_5,and most importantly no de caff tea bags,tea bags,no,caff tea bags,NaN,"[tea, tea bags]","[tea, bags]",None,"[tea, bags]",NaN,"[tea, tea bags]",tea bags
2087,GGL-20190685608_6,however overall a great stay for a decent price,stay,great,"stay, price",stay,"[stay, price]","[stay, price]",stay,"[stay, price]",stay,"[stay, price]",stay


In [217]:
df1['common'] = (df1['PerfectMatch'] + "," + df1['PerfectMatch_TB'] + "," + df1['PerfectMatch_Sp'] + "," + df1['PerfectMatch_EntityCleanupFile'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
w = pd.ExcelWriter('Final_entities.xlsx')

df.to_excel(w, sheet_name = 'All_final_entities', index = False)
df1.to_excel(w, sheet_name  = 'final_entities', index = False)

w.save()